In [35]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
k = 256
tree_depth = 6
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.214082717895508 | KNN Loss: 6.232590198516846 | BCE Loss: 1.9814924001693726
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.18209171295166 | KNN Loss: 6.232646942138672 | BCE Loss: 1.9494444131851196
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.180830955505371 | KNN Loss: 6.232651233673096 | BCE Loss: 1.948180079460144
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.165058135986328 | KNN Loss: 6.232604503631592 | BCE Loss: 1.9324537515640259
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.141905784606934 | KNN Loss: 6.232545852661133 | BCE Loss: 1.9093595743179321
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.150469779968262 | KNN Loss: 6.232403755187988 | BCE Loss: 1.9180659055709839
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.167644500732422 | KNN Loss: 6.232351779937744 | BCE Loss: 1.9352922439575195
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.160579681396484 | KNN Loss: 6.232356071472168 | BCE Loss: 1.9282238

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 7.382832050323486 | KNN Loss: 6.2128143310546875 | BCE Loss: 1.1700177192687988
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 7.365210056304932 | KNN Loss: 6.208114147186279 | BCE Loss: 1.1570959091186523
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 7.319311141967773 | KNN Loss: 6.20708703994751 | BCE Loss: 1.1122238636016846
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 7.334786415100098 | KNN Loss: 6.203920841217041 | BCE Loss: 1.1308658123016357
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 7.328578948974609 | KNN Loss: 6.200413227081299 | BCE Loss: 1.1281657218933105
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 7.3075175285339355 | KNN Loss: 6.196969509124756 | BCE Loss: 1.1105480194091797
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 7.304122447967529 | KNN Loss: 6.196120738983154 | BCE Loss: 1.1080015897750854
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 7.274090766906738 | KNN Loss: 6.193741321563721 | BCE Los

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 6.817303657531738 | KNN Loss: 5.757551193237305 | BCE Loss: 1.0597525835037231
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 6.8298659324646 | KNN Loss: 5.766357898712158 | BCE Loss: 1.0635080337524414
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 6.808271408081055 | KNN Loss: 5.757097244262695 | BCE Loss: 1.0511741638183594
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 6.804651737213135 | KNN Loss: 5.737254619598389 | BCE Loss: 1.067397117614746
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 6.8376383781433105 | KNN Loss: 5.74813985824585 | BCE Loss: 1.0894984006881714
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 6.7630462646484375 | KNN Loss: 5.725612640380859 | BCE Loss: 1.037433385848999
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 6.782855987548828 | KNN Loss: 5.726426124572754 | BCE Loss: 1.0564301013946533
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 6.758255958557129 | KNN Loss: 5.711081504821777 | BCE Loss: 

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 6.665799617767334 | KNN Loss: 5.621352672576904 | BCE Loss: 1.0444470643997192
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 6.676266670227051 | KNN Loss: 5.643748760223389 | BCE Loss: 1.0325177907943726
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 6.670083045959473 | KNN Loss: 5.616166591644287 | BCE Loss: 1.0539166927337646
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 6.766299247741699 | KNN Loss: 5.700475215911865 | BCE Loss: 1.0658239126205444
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 6.715966701507568 | KNN Loss: 5.669598579406738 | BCE Loss: 1.04636812210083
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 6.698056221008301 | KNN Loss: 5.666419982910156 | BCE Loss: 1.0316359996795654
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 6.733022212982178 | KNN Loss: 5.679352760314941 | BCE Loss: 1.0536694526672363
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 6.645362377166748 | KNN Loss: 5.630128383636475 | BCE Loss:

Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 6.698171615600586 | KNN Loss: 5.619533538818359 | BCE Loss: 1.0786380767822266
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 6.653359413146973 | KNN Loss: 5.622973442077637 | BCE Loss: 1.0303857326507568
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 6.652497291564941 | KNN Loss: 5.626805305480957 | BCE Loss: 1.025692105293274
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 6.656472206115723 | KNN Loss: 5.606506824493408 | BCE Loss: 1.0499656200408936
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 6.697007179260254 | KNN Loss: 5.646609783172607 | BCE Loss: 1.0503973960876465
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 6.674751281738281 | KNN Loss: 5.63383150100708 | BCE Loss: 1.0409200191497803
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 6.68321418762207 | KNN Loss: 5.612985134124756 | BCE Loss: 1.0702292919158936
Epoch 44 / 500 | iteration 15 / 30 | Total Loss: 6.666189193725586 | KNN Loss: 5.630638122558594 | BCE Loss: 

...

Epoch 448 / 500 | iteration 25 / 30 | Total Loss: 6.635524749755859 | KNN Loss: 5.599367618560791 | BCE Loss: 1.0361573696136475
Epoch   449: reducing learning rate of group 0 to 2.7058e-08.
Epoch 449 / 500 | iteration 0 / 30 | Total Loss: 6.635662078857422 | KNN Loss: 5.605652809143066 | BCE Loss: 1.0300092697143555
Epoch 449 / 500 | iteration 5 / 30 | Total Loss: 6.693984508514404 | KNN Loss: 5.647451400756836 | BCE Loss: 1.0465331077575684
Epoch 449 / 500 | iteration 10 / 30 | Total Loss: 6.695448398590088 | KNN Loss: 5.6347479820251465 | BCE Loss: 1.0607002973556519
Epoch 449 / 500 | iteration 15 / 30 | Total Loss: 6.683749198913574 | KNN Loss: 5.626462459564209 | BCE Loss: 1.0572868585586548
Epoch 449 / 500 | iteration 20 / 30 | Total Loss: 6.656353950500488 | KNN Loss: 5.63012170791626 | BCE Loss: 1.0262320041656494
Epoch 449 / 500 | iteration 25 / 30 | Total Loss: 6.671597480773926 | KNN Loss: 5.615428924560547 | BCE Loss: 1.056168794631958
Epoch 450 / 500 | iteration 0 / 30 | T

Epoch 459 / 500 | iteration 15 / 30 | Total Loss: 6.671774387359619 | KNN Loss: 5.628738880157471 | BCE Loss: 1.0430353879928589
Epoch 459 / 500 | iteration 20 / 30 | Total Loss: 6.710239410400391 | KNN Loss: 5.652767658233643 | BCE Loss: 1.0574719905853271
Epoch 459 / 500 | iteration 25 / 30 | Total Loss: 6.665363788604736 | KNN Loss: 5.596570014953613 | BCE Loss: 1.0687936544418335
Epoch 460 / 500 | iteration 0 / 30 | Total Loss: 6.672429084777832 | KNN Loss: 5.633646488189697 | BCE Loss: 1.0387825965881348
Epoch 460 / 500 | iteration 5 / 30 | Total Loss: 6.689088821411133 | KNN Loss: 5.6199631690979 | BCE Loss: 1.0691254138946533
Epoch 460 / 500 | iteration 10 / 30 | Total Loss: 6.671342849731445 | KNN Loss: 5.636999607086182 | BCE Loss: 1.0343433618545532
Epoch 460 / 500 | iteration 15 / 30 | Total Loss: 6.694055557250977 | KNN Loss: 5.662044048309326 | BCE Loss: 1.0320112705230713
Epoch 460 / 500 | iteration 20 / 30 | Total Loss: 6.6909894943237305 | KNN Loss: 5.671223163604736 | 

Epoch 470 / 500 | iteration 5 / 30 | Total Loss: 6.73414945602417 | KNN Loss: 5.693599700927734 | BCE Loss: 1.0405497550964355
Epoch 470 / 500 | iteration 10 / 30 | Total Loss: 6.708990573883057 | KNN Loss: 5.671413421630859 | BCE Loss: 1.0375771522521973
Epoch 470 / 500 | iteration 15 / 30 | Total Loss: 6.662715911865234 | KNN Loss: 5.614140510559082 | BCE Loss: 1.048575520515442
Epoch 470 / 500 | iteration 20 / 30 | Total Loss: 6.632256031036377 | KNN Loss: 5.601239204406738 | BCE Loss: 1.0310168266296387
Epoch 470 / 500 | iteration 25 / 30 | Total Loss: 6.696747303009033 | KNN Loss: 5.6356282234191895 | BCE Loss: 1.0611190795898438
Epoch 471 / 500 | iteration 0 / 30 | Total Loss: 6.6831254959106445 | KNN Loss: 5.640750885009766 | BCE Loss: 1.042374849319458
Epoch 471 / 500 | iteration 5 / 30 | Total Loss: 6.6779913902282715 | KNN Loss: 5.625983238220215 | BCE Loss: 1.052008032798767
Epoch 471 / 500 | iteration 10 / 30 | Total Loss: 6.68925666809082 | KNN Loss: 5.647762775421143 | BC

Epoch 480 / 500 | iteration 25 / 30 | Total Loss: 6.637829303741455 | KNN Loss: 5.596517562866211 | BCE Loss: 1.0413117408752441
Epoch 481 / 500 | iteration 0 / 30 | Total Loss: 6.676870822906494 | KNN Loss: 5.602390289306641 | BCE Loss: 1.0744805335998535
Epoch 481 / 500 | iteration 5 / 30 | Total Loss: 6.654966831207275 | KNN Loss: 5.598052024841309 | BCE Loss: 1.0569148063659668
Epoch 481 / 500 | iteration 10 / 30 | Total Loss: 6.650898456573486 | KNN Loss: 5.617271900177002 | BCE Loss: 1.033626675605774
Epoch 481 / 500 | iteration 15 / 30 | Total Loss: 6.660549163818359 | KNN Loss: 5.61375617980957 | BCE Loss: 1.046792984008789
Epoch 481 / 500 | iteration 20 / 30 | Total Loss: 6.6108293533325195 | KNN Loss: 5.593349933624268 | BCE Loss: 1.017479658126831
Epoch 481 / 500 | iteration 25 / 30 | Total Loss: 6.678884506225586 | KNN Loss: 5.632691383361816 | BCE Loss: 1.0461933612823486
Epoch 482 / 500 | iteration 0 / 30 | Total Loss: 6.627462863922119 | KNN Loss: 5.601944923400879 | BCE

Epoch 491 / 500 | iteration 15 / 30 | Total Loss: 6.679714202880859 | KNN Loss: 5.628486156463623 | BCE Loss: 1.0512281656265259
Epoch 491 / 500 | iteration 20 / 30 | Total Loss: 6.605490684509277 | KNN Loss: 5.593974590301514 | BCE Loss: 1.0115163326263428
Epoch 491 / 500 | iteration 25 / 30 | Total Loss: 6.646239757537842 | KNN Loss: 5.609392166137695 | BCE Loss: 1.0368475914001465
Epoch 492 / 500 | iteration 0 / 30 | Total Loss: 6.792389869689941 | KNN Loss: 5.7402024269104 | BCE Loss: 1.052187204360962
Epoch 492 / 500 | iteration 5 / 30 | Total Loss: 6.681451797485352 | KNN Loss: 5.624104022979736 | BCE Loss: 1.0573480129241943
Epoch 492 / 500 | iteration 10 / 30 | Total Loss: 6.635441780090332 | KNN Loss: 5.600375175476074 | BCE Loss: 1.0350664854049683
Epoch 492 / 500 | iteration 15 / 30 | Total Loss: 6.64028263092041 | KNN Loss: 5.606632232666016 | BCE Loss: 1.0336506366729736
Epoch 492 / 500 | iteration 20 / 30 | Total Loss: 6.675051689147949 | KNN Loss: 5.650125026702881 | BCE

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 2.8459,  3.5631,  2.5001,  3.1537,  3.3568,  0.6708,  2.3711,  2.0827,
          2.1993,  1.9800,  2.0103,  1.9847,  0.8091,  1.7728,  1.2656,  1.4792,
          2.4599,  2.8532,  2.6886,  2.2220,  1.6471,  2.9088,  2.0390,  2.3761,
          2.4719,  1.5146,  1.8861,  1.3733,  1.5239,  0.2685, -0.1907,  0.9555,
          0.2066,  0.9097,  1.4664,  1.3153,  0.9876,  3.2347,  0.8022,  1.3234,
          0.9608, -0.6582, -0.2546,  2.2673,  1.9412,  0.6751, -0.1418,  0.0990,
          1.4441,  2.3865,  1.7711,  0.0801,  1.4148,  0.4678, -0.5583,  1.0986,
          1.4249,  1.3179,  1.2942,  1.7721,  0.5641,  0.8216,  0.1511,  1.6695,
          1.2416,  1.6097, -1.7406,  0.2939,  2.2443,  2.0747,  2.4529,  0.4525,
          1.3193,  2.3935,  1.7749,  1.2830,  0.2490,  0.7379,  0.2319,  1.5311,
          0.0470,  0.3363,  1.7889, -0.3315,  0.2074, -1.0918, -2.2267, -0.2803,
          0.5183, -1.7710,  0.4681, -0.1521, -0.5530, -0.8952,  0.5180,  1.2434,
         -0.6065, -0.6744,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [10]:
dataset_ = [d[0].to('cpu') for d in dataset]

In [11]:
model = model.eval().to('cpu')
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:02<00:00,  5.63it/s]


In [41]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [48]:
clusters = DBSCAN(eps=0.02, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [49]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [51]:
tensor_dataset = torch.stack(dataset_)

In [52]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [53]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [54]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [55]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [56]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [57]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [58]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [59]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [60]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [61]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [62]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [63]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [64]:
losses = []
accs = []
sparsity = []

In [65]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
Epoch: 00 | Batch: 000 / 021 | Total loss: 9.760 | Reg loss: 0.007 | Tree loss: 9.760 | Accuracy: 0.000000 | 0.634 sec/iter
Epoch: 00 | Batch: 001 / 021 | Total loss: 9.752 | Reg loss: 0.007 | Tree loss: 9.752 | Accuracy: 0.000000 | 0.369 sec/iter
Epoch: 00 | Batch: 002 / 021 | Total loss: 9.742 | Reg loss: 0.007 | Tree loss: 9.742 | Accuracy: 0.000000 | 0.279 sec/iter
Epoch: 00 | Batch: 003 / 021 | Total loss: 9.728 | Reg loss: 0.006 | Tree loss: 9.728 | Accuracy: 0.000000 | 0.234 sec/iter
Epoch: 00 | Batch: 004 / 021 | Total loss: 9.717 | Reg loss: 0.006 | Tree loss: 9.717 | Accuracy: 0.000000 | 0.208 sec/iter
Epoch: 00 | Batch: 005 / 021 | Total loss: 9.703 | Reg loss: 0.006 | Tree loss: 9.703 | Accuracy: 0.000000 | 0.191 sec/iter
Epoch: 00 | Batch: 006 / 021 | Total loss: 9.693 | Reg loss: 0.006 | Tree loss: 9.693 | Accuracy: 0.000000 | 0.178 sec/iter
Epoch: 00 | Batch: 007 / 021 | Total loss: 

Epoch: 03 | Batch: 000 / 021 | Total loss: 9.402 | Reg loss: 0.006 | Tree loss: 9.402 | Accuracy: 0.000000 | 0.114 sec/iter
Epoch: 03 | Batch: 001 / 021 | Total loss: 9.391 | Reg loss: 0.006 | Tree loss: 9.391 | Accuracy: 0.000000 | 0.114 sec/iter
Epoch: 03 | Batch: 002 / 021 | Total loss: 9.380 | Reg loss: 0.007 | Tree loss: 9.380 | Accuracy: 0.001953 | 0.114 sec/iter
Epoch: 03 | Batch: 003 / 021 | Total loss: 9.366 | Reg loss: 0.007 | Tree loss: 9.366 | Accuracy: 0.000000 | 0.113 sec/iter
Epoch: 03 | Batch: 004 / 021 | Total loss: 9.349 | Reg loss: 0.007 | Tree loss: 9.349 | Accuracy: 0.003906 | 0.113 sec/iter
Epoch: 03 | Batch: 005 / 021 | Total loss: 9.341 | Reg loss: 0.007 | Tree loss: 9.341 | Accuracy: 0.011719 | 0.113 sec/iter
Epoch: 03 | Batch: 006 / 021 | Total loss: 9.331 | Reg loss: 0.007 | Tree loss: 9.331 | Accuracy: 0.011719 | 0.113 sec/iter
Epoch: 03 | Batch: 007 / 021 | Total loss: 9.315 | Reg loss: 0.008 | Tree loss: 9.315 | Accuracy: 0.023438 | 0.112 sec/iter
Epoch: 0

Epoch: 06 | Batch: 000 / 021 | Total loss: 9.043 | Reg loss: 0.011 | Tree loss: 9.043 | Accuracy: 0.376953 | 0.104 sec/iter
Epoch: 06 | Batch: 001 / 021 | Total loss: 9.036 | Reg loss: 0.011 | Tree loss: 9.036 | Accuracy: 0.345703 | 0.104 sec/iter
Epoch: 06 | Batch: 002 / 021 | Total loss: 9.022 | Reg loss: 0.011 | Tree loss: 9.022 | Accuracy: 0.349609 | 0.104 sec/iter
Epoch: 06 | Batch: 003 / 021 | Total loss: 8.997 | Reg loss: 0.011 | Tree loss: 8.997 | Accuracy: 0.414062 | 0.104 sec/iter
Epoch: 06 | Batch: 004 / 021 | Total loss: 8.988 | Reg loss: 0.011 | Tree loss: 8.988 | Accuracy: 0.371094 | 0.104 sec/iter
Epoch: 06 | Batch: 005 / 021 | Total loss: 8.987 | Reg loss: 0.011 | Tree loss: 8.987 | Accuracy: 0.330078 | 0.104 sec/iter
Epoch: 06 | Batch: 006 / 021 | Total loss: 8.973 | Reg loss: 0.011 | Tree loss: 8.973 | Accuracy: 0.335938 | 0.104 sec/iter
Epoch: 06 | Batch: 007 / 021 | Total loss: 8.956 | Reg loss: 0.012 | Tree loss: 8.956 | Accuracy: 0.353516 | 0.104 sec/iter
Epoch: 0

Epoch: 09 | Batch: 001 / 021 | Total loss: 8.653 | Reg loss: 0.015 | Tree loss: 8.653 | Accuracy: 0.386719 | 0.103 sec/iter
Epoch: 09 | Batch: 002 / 021 | Total loss: 8.640 | Reg loss: 0.015 | Tree loss: 8.640 | Accuracy: 0.367188 | 0.103 sec/iter
Epoch: 09 | Batch: 003 / 021 | Total loss: 8.628 | Reg loss: 0.015 | Tree loss: 8.628 | Accuracy: 0.341797 | 0.103 sec/iter
Epoch: 09 | Batch: 004 / 021 | Total loss: 8.610 | Reg loss: 0.015 | Tree loss: 8.610 | Accuracy: 0.392578 | 0.103 sec/iter
Epoch: 09 | Batch: 005 / 021 | Total loss: 8.588 | Reg loss: 0.015 | Tree loss: 8.588 | Accuracy: 0.380859 | 0.103 sec/iter
Epoch: 09 | Batch: 006 / 021 | Total loss: 8.590 | Reg loss: 0.015 | Tree loss: 8.590 | Accuracy: 0.355469 | 0.103 sec/iter
Epoch: 09 | Batch: 007 / 021 | Total loss: 8.575 | Reg loss: 0.015 | Tree loss: 8.575 | Accuracy: 0.341797 | 0.103 sec/iter
Epoch: 09 | Batch: 008 / 021 | Total loss: 8.558 | Reg loss: 0.016 | Tree loss: 8.558 | Accuracy: 0.365234 | 0.103 sec/iter
Epoch: 0

Epoch: 12 | Batch: 000 / 021 | Total loss: 8.283 | Reg loss: 0.018 | Tree loss: 8.283 | Accuracy: 0.396484 | 0.103 sec/iter
Epoch: 12 | Batch: 001 / 021 | Total loss: 8.252 | Reg loss: 0.018 | Tree loss: 8.252 | Accuracy: 0.398438 | 0.103 sec/iter
Epoch: 12 | Batch: 002 / 021 | Total loss: 8.250 | Reg loss: 0.018 | Tree loss: 8.250 | Accuracy: 0.375000 | 0.103 sec/iter
Epoch: 12 | Batch: 003 / 021 | Total loss: 8.225 | Reg loss: 0.018 | Tree loss: 8.225 | Accuracy: 0.359375 | 0.102 sec/iter
Epoch: 12 | Batch: 004 / 021 | Total loss: 8.202 | Reg loss: 0.018 | Tree loss: 8.202 | Accuracy: 0.373047 | 0.102 sec/iter
Epoch: 12 | Batch: 005 / 021 | Total loss: 8.204 | Reg loss: 0.018 | Tree loss: 8.204 | Accuracy: 0.353516 | 0.102 sec/iter
Epoch: 12 | Batch: 006 / 021 | Total loss: 8.177 | Reg loss: 0.018 | Tree loss: 8.177 | Accuracy: 0.369141 | 0.102 sec/iter
Epoch: 12 | Batch: 007 / 021 | Total loss: 8.175 | Reg loss: 0.018 | Tree loss: 8.175 | Accuracy: 0.333984 | 0.102 sec/iter
Epoch: 1

...

Epoch: 87 | Batch: 000 / 021 | Total loss: 1.850 | Reg loss: 0.037 | Tree loss: 1.850 | Accuracy: 0.427734 | 0.109 sec/iter
Epoch: 87 | Batch: 001 / 021 | Total loss: 1.905 | Reg loss: 0.037 | Tree loss: 1.905 | Accuracy: 0.437500 | 0.109 sec/iter
Epoch: 87 | Batch: 002 / 021 | Total loss: 1.794 | Reg loss: 0.037 | Tree loss: 1.794 | Accuracy: 0.437500 | 0.109 sec/iter
Epoch: 87 | Batch: 003 / 021 | Total loss: 1.759 | Reg loss: 0.037 | Tree loss: 1.759 | Accuracy: 0.408203 | 0.109 sec/iter
Epoch: 87 | Batch: 004 / 021 | Total loss: 1.750 | Reg loss: 0.037 | Tree loss: 1.750 | Accuracy: 0.392578 | 0.109 sec/iter
Epoch: 87 | Batch: 005 / 021 | Total loss: 1.764 | Reg loss: 0.037 | Tree loss: 1.764 | Accuracy: 0.431641 | 0.109 sec/iter
Epoch: 87 | Batch: 006 / 021 | Total loss: 1.771 | Reg loss: 0.037 | Tree loss: 1.771 | Accuracy: 0.447266 | 0.109 sec/iter
Epoch: 87 | Batch: 007 / 021 | Total loss: 1.713 | Reg loss: 0.037 | Tree loss: 1.713 | Accuracy: 0.437500 | 0.109 sec/iter
Epoch: 8

Epoch: 90 | Batch: 001 / 021 | Total loss: 1.767 | Reg loss: 0.037 | Tree loss: 1.767 | Accuracy: 0.414062 | 0.11 sec/iter
Epoch: 90 | Batch: 002 / 021 | Total loss: 1.775 | Reg loss: 0.037 | Tree loss: 1.775 | Accuracy: 0.449219 | 0.11 sec/iter
Epoch: 90 | Batch: 003 / 021 | Total loss: 1.748 | Reg loss: 0.037 | Tree loss: 1.748 | Accuracy: 0.412109 | 0.11 sec/iter
Epoch: 90 | Batch: 004 / 021 | Total loss: 1.751 | Reg loss: 0.037 | Tree loss: 1.751 | Accuracy: 0.388672 | 0.11 sec/iter
Epoch: 90 | Batch: 005 / 021 | Total loss: 1.683 | Reg loss: 0.037 | Tree loss: 1.683 | Accuracy: 0.429688 | 0.11 sec/iter
Epoch: 90 | Batch: 006 / 021 | Total loss: 1.685 | Reg loss: 0.037 | Tree loss: 1.685 | Accuracy: 0.410156 | 0.11 sec/iter
Epoch: 90 | Batch: 007 / 021 | Total loss: 1.696 | Reg loss: 0.037 | Tree loss: 1.696 | Accuracy: 0.412109 | 0.11 sec/iter
Epoch: 90 | Batch: 008 / 021 | Total loss: 1.671 | Reg loss: 0.037 | Tree loss: 1.671 | Accuracy: 0.402344 | 0.11 sec/iter
Epoch: 90 | Batc

Epoch: 93 | Batch: 002 / 021 | Total loss: 1.714 | Reg loss: 0.037 | Tree loss: 1.714 | Accuracy: 0.419922 | 0.11 sec/iter
Epoch: 93 | Batch: 003 / 021 | Total loss: 1.696 | Reg loss: 0.037 | Tree loss: 1.696 | Accuracy: 0.455078 | 0.11 sec/iter
Epoch: 93 | Batch: 004 / 021 | Total loss: 1.755 | Reg loss: 0.037 | Tree loss: 1.755 | Accuracy: 0.441406 | 0.11 sec/iter
Epoch: 93 | Batch: 005 / 021 | Total loss: 1.711 | Reg loss: 0.037 | Tree loss: 1.711 | Accuracy: 0.406250 | 0.11 sec/iter
Epoch: 93 | Batch: 006 / 021 | Total loss: 1.679 | Reg loss: 0.037 | Tree loss: 1.679 | Accuracy: 0.439453 | 0.11 sec/iter
Epoch: 93 | Batch: 007 / 021 | Total loss: 1.700 | Reg loss: 0.037 | Tree loss: 1.700 | Accuracy: 0.402344 | 0.11 sec/iter
Epoch: 93 | Batch: 008 / 021 | Total loss: 1.642 | Reg loss: 0.037 | Tree loss: 1.642 | Accuracy: 0.423828 | 0.11 sec/iter
Epoch: 93 | Batch: 009 / 021 | Total loss: 1.687 | Reg loss: 0.037 | Tree loss: 1.687 | Accuracy: 0.384766 | 0.11 sec/iter
Epoch: 93 | Batc

Epoch: 96 | Batch: 003 / 021 | Total loss: 1.697 | Reg loss: 0.036 | Tree loss: 1.697 | Accuracy: 0.443359 | 0.11 sec/iter
Epoch: 96 | Batch: 004 / 021 | Total loss: 1.661 | Reg loss: 0.036 | Tree loss: 1.661 | Accuracy: 0.425781 | 0.11 sec/iter
Epoch: 96 | Batch: 005 / 021 | Total loss: 1.644 | Reg loss: 0.036 | Tree loss: 1.644 | Accuracy: 0.417969 | 0.11 sec/iter
Epoch: 96 | Batch: 006 / 021 | Total loss: 1.612 | Reg loss: 0.036 | Tree loss: 1.612 | Accuracy: 0.417969 | 0.11 sec/iter
Epoch: 96 | Batch: 007 / 021 | Total loss: 1.633 | Reg loss: 0.036 | Tree loss: 1.633 | Accuracy: 0.439453 | 0.11 sec/iter
Epoch: 96 | Batch: 008 / 021 | Total loss: 1.629 | Reg loss: 0.036 | Tree loss: 1.629 | Accuracy: 0.453125 | 0.11 sec/iter
Epoch: 96 | Batch: 009 / 021 | Total loss: 1.643 | Reg loss: 0.036 | Tree loss: 1.643 | Accuracy: 0.435547 | 0.11 sec/iter
Epoch: 96 | Batch: 010 / 021 | Total loss: 1.596 | Reg loss: 0.036 | Tree loss: 1.596 | Accuracy: 0.386719 | 0.11 sec/iter
Epoch: 96 | Batc

Epoch: 99 | Batch: 003 / 021 | Total loss: 1.633 | Reg loss: 0.036 | Tree loss: 1.633 | Accuracy: 0.445312 | 0.11 sec/iter
Epoch: 99 | Batch: 004 / 021 | Total loss: 1.672 | Reg loss: 0.036 | Tree loss: 1.672 | Accuracy: 0.439453 | 0.11 sec/iter
Epoch: 99 | Batch: 005 / 021 | Total loss: 1.665 | Reg loss: 0.036 | Tree loss: 1.665 | Accuracy: 0.423828 | 0.11 sec/iter
Epoch: 99 | Batch: 006 / 021 | Total loss: 1.574 | Reg loss: 0.036 | Tree loss: 1.574 | Accuracy: 0.441406 | 0.11 sec/iter
Epoch: 99 | Batch: 007 / 021 | Total loss: 1.611 | Reg loss: 0.036 | Tree loss: 1.611 | Accuracy: 0.394531 | 0.11 sec/iter
Epoch: 99 | Batch: 008 / 021 | Total loss: 1.561 | Reg loss: 0.036 | Tree loss: 1.561 | Accuracy: 0.453125 | 0.11 sec/iter
Epoch: 99 | Batch: 009 / 021 | Total loss: 1.655 | Reg loss: 0.036 | Tree loss: 1.655 | Accuracy: 0.394531 | 0.11 sec/iter
Epoch: 99 | Batch: 010 / 021 | Total loss: 1.536 | Reg loss: 0.036 | Tree loss: 1.536 | Accuracy: 0.451172 | 0.11 sec/iter
Epoch: 99 | Batc

In [66]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [68]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 5.393939393939394


# Extract Rules

# Accumulate samples in the leaves

In [69]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 33


In [70]:
method = 'greedy'

In [71]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/miniconda3/envs/rambo/lib/python3.8/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [72]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))


============== Pattern 1 ==============
============== Pattern 2 ==============
8869
============== Pattern 3 ==============
542
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
211
============== Pattern 21 ==============
749
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
========